# **Fake News Detection from Tweets**

Task: To explore various machine learning models to select the best model for classifying fake news tweets.

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

train_df = pd.read_csv("FakeNews_train.csv")
test_df = pd.read_csv("FakeNews_test.csv")

train_df.shape, test_df.shape

((5136, 2), (1284, 2))

In [2]:
# Sanity Check
train_df.head()
test_df.head()

,tweet,label
0,• Businesses and offices can reopen for staff ...,real
1,RT @CDCDirector: We do not know yet if the ant...,real
2,This is an image of a suspected coronavirus va...,fake
3,We can’t forget that in the middle of a global...,fake
4,#IndiaFightsCorona Focused and effective effor...,real


,tweet,label
0,"Experts Call Out Claims That Cow Dung/Urine, Y...",fake
1,Bill Gates predicted and simulated the COVID-1...,fake
2,Global coronavirus deaths exceed 800000 https:...,fake
3,An Ayurveda practitioner Devender Sharma gave ...,fake
4,Meghan Markle Planning Huge $200K Birthday Par...,fake


## Data Preprocessing

Here I apply appropriate preprocessing to the tweets

*   Clean the text data by removing URLs, hashtags, and mentions.
*   Tokenize the text data and remove stopwords
*   Lemmatize the text data.

I apply the above preprocessing to both train and test tweets.



In [3]:
import re
import nltk


# Function to clean tweets (remove URLS, mentions, hastags)
def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    return text


tweet_list = list(train_df["tweet"])
cleaned_tweets = [clean_tweet(tweet) for tweet in tweet_list]
cleaned_tweets[:5]

['• Businesses and offices can reopen for staff and customers. Services can be provided in peoples’ homes. • Hairdressers and beauticians can reopen but must wear PPE. • Hospitality businesses can reopen but patrons must be seated separate and have single servers.',
 'RT : We do not know yet if the antibodies can protect you from  reinfection. Regardless of your antibody test results…',
 'This is an image of a suspected coronavirus vaccine causing COVID-19.',
 "We can’t forget that in the middle of a global pandemic, the Trump Administration is trying to gut Obamacare and rip health insurance away from millions. It's morally reprehensible.",
 ' Focused and effective efforts of containment testing isolation and treatment have resulted in increasing percentage recovery rates and steadily falling percentage active cases. ']

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def tokenize_and_remove_stopwords(text):
  tokenized_text = word_tokenize(text)
  filtered_text = [token for token in tokenized_text if token not in stop_words]
  return " ".join(filtered_text)

tokenized_tweets = [tokenize_and_remove_stopwords(tweet) for tweet in cleaned_tweets]
tokenized_tweets[:5]

['• Businesses offices reopen staff customers . Services provided peoples ’ homes . • Hairdressers beauticians reopen must wear PPE . • Hospitality businesses reopen patrons must seated separate single servers .',
 'RT : We know yet antibodies protect reinfection . Regardless antibody test results…',
 'This image suspected coronavirus vaccine causing COVID-19 .',
 "We ’ forget middle global pandemic , Trump Administration trying gut Obamacare rip health insurance away millions . It 's morally reprehensible .",
 'Focused effective efforts containment testing isolation treatment resulted increasing percentage recovery rates steadily falling percentage active cases .']

In [6]:
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

def lemmatize_text(text):
  tokens = word_tokenize(text)
  lemmas = [wnl.lemmatize(token) for token in tokens]
  return " ".join(lemmas)

lemmatized_tweets = [lemmatize_text(tweet) for tweet in tokenized_tweets]
lemmatized_tweets[:5]

['• Businesses office reopen staff customer . Services provided people ’ home . • Hairdressers beautician reopen must wear PPE . • Hospitality business reopen patron must seated separate single server .',
 'RT : We know yet antibody protect reinfection . Regardless antibody test results…',
 'This image suspected coronavirus vaccine causing COVID-19 .',
 "We ’ forget middle global pandemic , Trump Administration trying gut Obamacare rip health insurance away million . It 's morally reprehensible .",
 'Focused effective effort containment testing isolation treatment resulted increasing percentage recovery rate steadily falling percentage active case .']

In [7]:
preprocessed_tweets = lemmatized_tweets
train_df["tweet"] = preprocessed_tweets
train_df.head()

,tweet,label
0,• Businesses office reopen staff customer . Se...,real
1,RT : We know yet antibody protect reinfection ...,real
2,This image suspected coronavirus vaccine causi...,fake
3,"We ’ forget middle global pandemic , Trump Adm...",fake
4,Focused effective effort containment testing i...,real


In [8]:
# Apply preprocessing to test_df
test_tweet_list = list(test_df["tweet"])
test_cleaned_tweets = [clean_tweet(tweet) for tweet in test_tweet_list]
test_tokenized_tweets = [tokenize_and_remove_stopwords(tweet) for tweet in test_cleaned_tweets]
test_preprocessed_tweets = [lemmatize_text(tweet) for tweet in test_tokenized_tweets]

test_df["tweet"] = test_preprocessed_tweets
test_df.head()

,tweet,label
0,"Experts Call Out Claims That Cow Dung/Urine , ...",fake
1,Bill Gates predicted simulated COVID-19 pandem...,fake
2,Global coronavirus death exceed 800000,fake
3,An Ayurveda practitioner Devender Sharma gave ...,fake
4,Meghan Markle Planning Huge $ 200K Birthday Pa...,fake


## Defining ML models

Here I will train and test the following models:


*   Logistic Regression
*   SVC
*   MLP Classifier



In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


def train_and_evaluate_classifier(classifier, X_train, y_actual, X_test, y_test_actual):
  classifier.fit(X_train, y_actual)
  y_pred = classifier.predict(X_test)
  accuracy = accuracy_score(y_test_actual, y_pred)
  return accuracy


# Extract text and labels
X_train = train_df['tweet']
y_train = train_df['label']
X_test = test_df['tweet']
y_test = test_df['label']

## Exploring Basic Text Features (TF-IDF)

For each tweet, I extract TF-IDF features - with the max_features parameter set to 5000 - and compute the accuracy using the three ML models.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a CountVectorizer for unigrams (bag of words)
vectorizer = TfidfVectorizer(max_features=5000)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [11]:
lr_classifier = LogisticRegression()
lr_accuracy = train_and_evaluate_classifier(lr_classifier, X_train_vec, y_train, X_test_vec, y_test)
print (f"Accuracy of Logistic Regression = {lr_accuracy*100}%")

Accuracy of Logistic Regression = 91.66666666666666%


In [12]:
svc_classifier = SVC(kernel="linear")
svc_accuracy = train_and_evaluate_classifier(svc_classifier, X_train_vec, y_train, X_test_vec, y_test)
print (f"Accuracy of SVC = {svc_accuracy*100}%")

Accuracy of SVC = 92.601246105919%


In [13]:
mlp_classifier = MLPClassifier(random_state=1, max_iter=100)
mlp_accuracy = train_and_evaluate_classifier(mlp_classifier, X_train_vec, y_train, X_test_vec, y_test)
print (f"Accuracy of MLP = {mlp_accuracy*100}%")

Accuracy of MLP = 90.57632398753894%


After training and testing the three models, the results are as follows:


1.   LogisticRegression accuracy - 91.66%
2.   SVC with linear kernel - 92.60%
3.   MLP Classifier - 90.58%

Based on the results, the best model is SVC with an accuracy of 92.6%. The consistent performance across the three models also indicates that extracting TF-IDF features is a good set of features for this problem.


In [14]:
# Print 10 examples from the test data
y_pred = svc_classifier.predict(X_test_vec)
print("\n10 examples from the test data:")
print("Index | Actual Label | Predicted Label | Tweet")
print("-" * 70)

for i in range(10):
    print(f"{i+1}     | {y_test.iloc[i]}\t     | {y_pred[i]}\t       | {X_test.iloc[i]}")


10 examples from the test data:
Index | Actual Label | Predicted Label | Tweet
----------------------------------------------------------------------
1     | fake	     | fake	       | Experts Call Out Claims That Cow Dung/Urine , Yoga , AYUSH Can Prevent Or Treat COVID-19
2     | fake	     | fake	       | Bill Gates predicted simulated COVID-19 pandemic .
3     | fake	     | fake	       | Global coronavirus death exceed 800000
4     | fake	     | fake	       | An Ayurveda practitioner Devender Sharma gave false COVID-19 positive report 125 patient contracted virus , killed kidney .
5     | fake	     | fake	       | Meghan Markle Planning Huge $ 200K Birthday Party During Coronavirus Pandemic ?
6     | real	     | real	       | National Expert Group Vaccine Administration meet Domestic Vaccine Manufactures
7     | fake	     | fake	       | I cited retrovirus journal written professor . Flu vaccine contain coronaviruses would kill dog cell line . Judy Mikovits ' claim complete medical m

From the above 10 examples from the test data, there are no false positve or negative tweets: the model correctly predicts the labels (real/fake) for these 10 examples.

## Exploring Averaged Word Embeddings as Features

For each tweet, I extract Averaged Word Embeddings using the word2vec model and use them as features. I then compute the accuracy using the three ML models.

In [15]:
import numpy as np
import gensim.downloader as api
word2vec_model = api.load("word2vec-google-news-300")

def get_tweet_embedding(tweet, word2vec_model):
    words = word_tokenize(tweet)
    embeddings = [word2vec_model[word] for word in words if word in word2vec_model]
    if len(embeddings) == 0:
        return [0] * 300
    return np.mean(embeddings, axis=0)

# Extract averaged word embeddings for the training and test sets
X_train_emb = [get_tweet_embedding(tweet, word2vec_model) for tweet in X_train]
X_test_emb = [get_tweet_embedding(tweet, word2vec_model) for tweet in X_test]

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [16]:
# Train and evaluate the three ML models
lr_classifier = LogisticRegression()
lr_accuracy = train_and_evaluate_classifier(lr_classifier, X_train_emb, y_train, X_test_emb, y_test)
print(f"Accuracy of Logistic Regression = {lr_accuracy*100}%")

svc_classifier = SVC()
svc_accuracy = train_and_evaluate_classifier(svc_classifier, X_train_emb, y_train, X_test_emb, y_test)
print(f"Accuracy of SVC = {svc_accuracy*100}%")

mlp_classifier = MLPClassifier()
mlp_accuracy = train_and_evaluate_classifier(mlp_classifier, X_train_emb, y_train, X_test_emb, y_test)
print(f"Accuracy of MLP Classifier = {mlp_accuracy*100}%")

Accuracy of Logistic Regression = 90.03115264797508%
Accuracy of SVC = 92.21183800623052%
Accuracy of MLP Classifier = 91.43302180685359%


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [17]:
# Print 10 examples from the test data
y_pred = svc_classifier.predict(X_test_emb)
print("\n10 examples from the test data:")
print("Index | Actual Label | Predicted Label | Tweet")
print("-" * 70)

for i in range(10):
    print(f"{i+1}     | {y_test.iloc[i]}\t     | {y_pred[i]}\t       | {X_test.iloc[i]}")


10 examples from the test data:
Index | Actual Label | Predicted Label | Tweet
----------------------------------------------------------------------
1     | fake	     | fake	       | Experts Call Out Claims That Cow Dung/Urine , Yoga , AYUSH Can Prevent Or Treat COVID-19
2     | fake	     | fake	       | Bill Gates predicted simulated COVID-19 pandemic .
3     | fake	     | fake	       | Global coronavirus death exceed 800000
4     | fake	     | fake	       | An Ayurveda practitioner Devender Sharma gave false COVID-19 positive report 125 patient contracted virus , killed kidney .
5     | fake	     | fake	       | Meghan Markle Planning Huge $ 200K Birthday Party During Coronavirus Pandemic ?
6     | real	     | fake	       | National Expert Group Vaccine Administration meet Domestic Vaccine Manufactures
7     | fake	     | fake	       | I cited retrovirus journal written professor . Flu vaccine contain coronaviruses would kill dog cell line . Judy Mikovits ' claim complete medical m

Based on the results above, the Support Vector Classifier (SVC) model appears to be the best performer, achieving an accuracy of 92.21%. The higher accuracy of the SVC model suggests that the averaged word embeddings extracted from the pre-trained word2vec model were effective features for this classification task, and the SVC was able to leverage these features to make more accurate predictions than the other models. The consistent performance across the three models also indicates that the averaged word embeddings are a reasonably robust set of features for this problem.

## Exploring Sentence Transformer Embeddings as Features

For each tweet, I extract Sentence Transformer Embeddings using the BERT model and use them as features. I then compute the accuracy using the three ML models.

In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained BERT model
model = SentenceTransformer('bert-base-uncased')

# Extract sentence embeddings for the training and test sets
X_train_emb = [model.encode(tweet) for tweet in X_train]
X_test_emb = [model.encode(tweet) for tweet in X_test]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [21]:
# Train and evaluate the three ML models
lr_classifier = LogisticRegression()
lr_accuracy = train_and_evaluate_classifier(lr_classifier, X_train_emb, y_train, X_test_emb, y_test)
print(f"Accuracy of Logistic Regression = {lr_accuracy*100}%")

svc_classifier = SVC()
svc_accuracy = train_and_evaluate_classifier(svc_classifier, X_train_emb, y_train, X_test_emb, y_test)
print(f"Accuracy of SVC = {svc_accuracy*100}%")

mlp_classifier = MLPClassifier()
mlp_accuracy = train_and_evaluate_classifier(mlp_classifier, X_train_emb, y_train, X_test_emb, y_test)
print(f"Accuracy of MLP Classifier = {mlp_accuracy*100}%")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of Logistic Regression = 92.13395638629284%
Accuracy of SVC = 91.82242990654206%
Accuracy of MLP Classifier = 92.44548286604362%


In [22]:
# Print 10 examples from the test data
y_pred = mlp_classifier.predict(X_test_emb)
print("\n10 examples from the test data:")
print("Index | Actual Label | Predicted Label | Tweet")
print("-" * 70)

for i in range(10):
    print(f"{i+1}     | {y_test.iloc[i]}\t     | {y_pred[i]}\t       | {X_test.iloc[i]}")


10 examples from the test data:
Index | Actual Label | Predicted Label | Tweet
----------------------------------------------------------------------
1     | fake	     | fake	       | Experts Call Out Claims That Cow Dung/Urine , Yoga , AYUSH Can Prevent Or Treat COVID-19
2     | fake	     | fake	       | Bill Gates predicted simulated COVID-19 pandemic .
3     | fake	     | fake	       | Global coronavirus death exceed 800000
4     | fake	     | fake	       | An Ayurveda practitioner Devender Sharma gave false COVID-19 positive report 125 patient contracted virus , killed kidney .
5     | fake	     | fake	       | Meghan Markle Planning Huge $ 200K Birthday Party During Coronavirus Pandemic ?
6     | real	     | real	       | National Expert Group Vaccine Administration meet Domestic Vaccine Manufactures
7     | fake	     | fake	       | I cited retrovirus journal written professor . Flu vaccine contain coronaviruses would kill dog cell line . Judy Mikovits ' claim complete medical m

The results show that using sentence transformer embeddings as features extracted from a pre-trained BERT-based model leads to improved performance compared to the previous approach using averaged word embeddings. The MLP Classifier achieves the highest accuracy at 92.99%. This suggests the MLP's ability to capture complex relationships in the sentence-level features is well-suited for this classification task. The consistent high performance across the three models also demonstrates the robustness of the sentence embeddings as features.